# 🖼️ 1. Datensatz vorbereiten

Jede Gruppe erhält einen Ordner mit **Rohbildern**. Diese sollen in [Makesense.ai](https://www.makesense.ai) annotiert werden.

## 🔧 Schritt 1: Annotation mit Makesense.ai
1. Öffnet [https://www.makesense.ai](https://www.makesense.ai)
2. Bilder hochladen
3. Modus: **Segmentation**
4. Klasse: z. B. `baum`, `überdachung`
5. Export: Format `Pascal VOC Segmentation (PNG masks)`

👉 Jetzt bitte eure annotierten Masken und Bilder wieder hochladen. Verwende folgenden Platzhalter:

```python
from IPython.display import display, Image
# Ersetze mit deinem Bildpfad nach dem Upload
display(Image("beispielbild.jpg"))
```

---

## 🔁 Schritt 2: Daten augmentieren

```python
import albumentations as A
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=45, p=0.5),
])

def augment_pair(image_path, mask_path):
    image = np.array(Image.open(image_path))
    mask = np.array(Image.open(mask_path))

    augmented = transform(image=image, mask=mask)
    aug_img = augmented["image"]
    aug_mask = augmented["mask"]

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(aug_img)
    axs[0].set_title("Bild")
    axs[1].imshow(aug_mask, cmap='gray')
    axs[1].set_title("Maske")
    plt.show()
```

---

## 🔗 Schritt 3: Datensätze zusammenführen

```python
import shutil
from glob import glob
import os

teilnehmer_ordner = ["gruppe1", "gruppe2", "gruppe3"]  # Anpassen!

for ordner in teilnehmer_ordner:
    for datei in glob(f"{ordner}/images/*.jpg"):
        shutil.copy(datei, "gesamt/images/")
    for datei in glob(f"{ordner}/masks/*.png"):
        shutil.copy(datei, "gesamt/masks/")
```

Jetzt per `sklearn` aufteilen:

```python
from sklearn.model_selection import train_test_split

images = sorted(glob("gesamt/images/*.jpg"))
masks = sorted(glob("gesamt/masks/*.png"))

train_imgs, val_imgs, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

print(f"Train: {len(train_imgs)}, Validation: {len(val_imgs)}")
```